In [ ]:
from openai import OpenAI

In [ ]:
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

# Step 1 : Import neccessary libraries

In [ ]:
import os
#from constants import openai_key
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

from langchain.memory import ConversationBufferMemory

from langchain.chains import SequentialChain

# Step 2: Start local LLM server , and point client to base url of the existing server 

In [ ]:
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")

# Step 3:Define roles to the Conversational system for what purpose one is going to utilize , in a way tweaking the system prompt content.

In [ ]:
result = client.chat.completions.create(
  model="local-model", # this field is currently unused
  messages=[
    #{"role": "system", "content": "Do the name entity tagging in this format Product: [PRODUCT] Time Period: [TIME_PERIOD] Month: [MONTH] Data: [DATA] Chart Type: [CHART_TYPE]"},
    {"role": "system", "content": "do the name entity tagging  (Product: [PRODUCT] Time Period: [TIME_PERIOD] Month: [MONTH] Data: [DATA] Chart Type: [CHART_TYPE])"},
    {"role": "user", "content": user_input}
  ],
  temperature=0.7,
)

In [ ]:
user_input

In [ ]:
print(result.choices[0].message)

# user can also play with its parameter to set the output query response like shown below

In [ ]:
import openai

# Set your OpenAI GPT-3 API key
openai = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")


# Example user query
user_query = "Show me last 4 months sale of product iphone as bar chart."

# Structure the prompt to get named entity tagging format
prompt = f"User Query: {user_query}\nResponse: Product: [PRODUCT] Time Period: [TIME_PERIOD] Month: [MONTH] Data: [DATA] Chart Type: [CHART_TYPE]"

# Generate response
response = openai.Completion.create(
    engine="text-davinci-003",  # You can experiment with different engines
    prompt=prompt,
    max_tokens=200,
    temperature=0.7,
)

# Extract information from the generated response
extracted_info = response['choices'][0]['text'].strip()

# Print the extracted information
print(extracted_info)

# Now let's create a chains of prompts which take input and returns output in a chain of different scenario's !

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")

In [ ]:
## Integrate our code OpenAI API
import os
from constants import openai_key
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

from langchain.memory import ConversationBufferMemory

from langchain.chains import SequentialChain

import streamlit as st

os.environ["OPENAI_API_KEY"]=openai_key

# streamlit framework

st.title('STmicroelectronics')
input_text=st.text_input("Search whatever you want ?")

# Prompt Templates

first_input_prompt=PromptTemplate(
    input_variables=['name'],
    template="is {name} a good person?"
)

# Memory

person_memory = ConversationBufferMemory(input_key='name', memory_key='chat_history')
dob_memory = ConversationBufferMemory(input_key='person', memory_key='chat_history')
descr_memory = ConversationBufferMemory(input_key='dob', memory_key='description_history')

## OPENAI LLMS
llm=OpenAI(temperature=0.8)
chain=LLMChain(
    llm=llm,prompt=first_input_prompt,verbose=True,output_key='person',memory=person_memory)

# Prompt Templates

second_input_prompt=PromptTemplate(
    input_variables=['person'],
    template="tell about whether this {person} "
)

chain2=LLMChain(
    llm=llm,prompt=second_input_prompt,verbose=True,output_key='dob',memory=dob_memory)
# Prompt Templates

third_input_prompt=PromptTemplate(
    input_variables=['dob'],
    template="Mention 5 major events happened around {dob} in the world"
)
chain3=LLMChain(llm=llm,prompt=third_input_prompt,verbose=True,output_key='description',memory=descr_memory)
parent_chain=SequentialChain(
    chains=[chain,chain2,chain3],input_variables=['name'],output_variables=['person','dob','description'],verbose=True)



if input_text:
    st.write(parent_chain({'name':input_text}))

    with st.expander('Person Name'): 
        st.info(person_memory.buffer)

    with st.expander('Major Events'): 
        st.info(descr_memory.buffer)